**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [15]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

#группируем по пользователям
c_ratings = ratings.groupby('userId').count()

#отфильтровываем пользователей, у которых больше 100 оценок
r_filt = c_ratings[c_ratings.rating > 100]

#ищем в исходной таблице минимальные и максимальные значения времени, записываем их в отдельную таблицу
minmax = ratings.groupby('userId').agg(['min', 'max'])['timestamp']

#соединяем отфильрованных пользователей с полученный таблицей 
t_new = r_filt.join(minmax, how='left', on=['userId'])

#создаем столбец с временем жизни
t_new['lifetime'] = (t_new['max'] - t_new['min'])

#сортируем по времени жизни и оставляем только этот столбец для показа
t_new.sort_values('lifetime', ascending=False).filter(items = ['lifetime']).head()


,lifetime
userId,
380,515789661
547,501810535
427,476569256
15,471393496
624,457492507


In [16]:
# среднее время жизни
t_new.lifetime.mean()

40080507.4496124

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [9]:
# с указанием адреса
client_base.merge(rzd, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(air, how='left', on='client_id').fillna(0)


,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [10]:
# без указания адреса
auto.merge(rzd, how='outer', on='client_id').merge(air, how='outer', on='client_id').fillna(0)

,client_id,auto_revenue,rzd_revenue,air_revenue
0,113,57483.0,10283.0,0.0
1,114,83.0,5774.0,0.0
2,115,912.0,981.0,81.0
3,116,4834.0,0.0,4.0
4,117,98.0,0.0,13.0
5,111,0.0,1093.0,0.0
6,112,0.0,2810.0,0.0
7,118,0.0,0.0,173.0
